<a href="https://colab.research.google.com/github/gkondayya/Options/blob/main/RealtimeStockNewsSentimentAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request

In [3]:
def get_ticks(url):
    req = Request(url=url,headers={"User-Agent": "Chrome"}) 
    response = urlopen(req)    
    html = BeautifulSoup(response,"html.parser")
    news_table = html.find(class_ = 'page')
    
    ticks = list()
    stocks = list()
    for name_box in news_table.find_all('a', href = True):
        stocks.append(name_box.text.strip())
        ticks.append(name_box['href'].split('-')[-1].strip())
    d = {'stock':stocks, 'tick':ticks}
    df = pd.DataFrame(data = d)
    return df

In [4]:
url = "https://www.tickertape.in/stocks"

tick_df = get_ticks(url)

In [5]:
tick_df

,stock,tick
0,ABB India Ltd,ABB
1,Adani Enterprises Ltd,ADEL
2,Adani Green Energy Ltd,ADNA
3,Adani Ports and Special Economic Zone Ltd,APSE
4,Adani Power Ltd,ADAN
...,...,...
95,United Spirits Ltd,UNSP
96,Varun Beverages Ltd,VARB
97,Vedanta Ltd,VDAN
98,Wipro Ltd,WIPR


In [6]:
def get_data(url):
    req = Request(url=url,headers={"User-Agent": "Chrome"}) 
    response = urlopen(req).read()    
    html = BeautifulSoup(response,"html.parser")
    news_table = html.find(class_ = 'latest-news-holder')
    
    news = list()
    
    for name_box in news_table.find_all('p', class_='shave-root'):
        news.append(name_box.text.strip())
    
    return news

In [7]:
news = list()
for i, tick in enumerate(tick_df['tick']):
    url = "https://www.tickertape.in/stocks/" + tick + "/news?checklist=basic&ref=stock-overview_overview-sections&type=news"
    headlines = get_data(url)
    news.append(headlines)

In [8]:
for i, n in enumerate(news):
    print(i+1, ")", n)
    print('-------')

1 ) ['Trade setup for today: Top 15 things to know before the opening bell', 'ABB to acquire controlling\xa0stake in EV charging platform Numocity', 'ABB’s e-mobility unit hikes stake in start-up Numocity']
-------
2 ) ['Adani Enterprises seen replacing Shree Cement in Nifty 50 index', 'Stocks in news: LIC, Nykaa, IRCTC, Adani Enterprises and more', 'Stocks that will see action today May 30']
-------
3 ) ['Stocks that will see action today May 30', 'Adani Green Energy commissions wind-solar hybrid power plant in Rajasthan', 'Adani Green Energy arm commissions India’s first wind-solar hybrid plant in Rajasthan']
-------
4 ) ['Buy Adani Ports and SEZ ; target of Rs 900: ICICI Direct', 'Adani Ports Q4 Review - Embarking On Another Round Of Capex: ICICI Direct', 'Adani Ports Q4: Net profit falls 21% to Rs 1,033 cr; EBITDA up 22%']
-------
5 ) ['Power ministry earmarks 81 thermal units to move coal to renewable by 2026', 'These BSE 500 firms posted up to 35,281%\xa0YoY rise in Q4 net profit

In [12]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 25.7 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 596 kB 48.6 MB/s 
     |████████████████████████████████| 6.6 MB 26.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [13]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

Downloading:   0%|          | 0.00/533 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/221k [00:00<?, ?B/s]

In [14]:
labels = {0:'neutral', 1:'positive',2:'negative'}

In [15]:
def detect(news):
    tot_val = list()
    for n in news:
        if len(n) == 0:
            tot_val.append(['neutral'])
        else:
            inputs = tokenizer(n, return_tensors="pt", padding=True)
            outputs = finbert(**inputs)[0]
            val = list()
            for idx, sent in enumerate(n):
                val.append(labels[np.argmax(outputs.detach().numpy()[idx])])
                print(sent, '----', val[idx])
            print('#######################################################')    
            tot_val.append(val)
    return tot_val

In [16]:
tot_val = detect(news)

print(tot_val)

Trade setup for today: Top 15 things to know before the opening bell ---- neutral
ABB to acquire controlling stake in EV charging platform Numocity ---- neutral
ABB’s e-mobility unit hikes stake in start-up Numocity ---- neutral
#######################################################
Adani Enterprises seen replacing Shree Cement in Nifty 50 index ---- neutral
Stocks in news: LIC, Nykaa, IRCTC, Adani Enterprises and more ---- neutral
Stocks that will see action today May 30 ---- neutral
#######################################################
Stocks that will see action today May 30 ---- neutral
Adani Green Energy commissions wind-solar hybrid power plant in Rajasthan ---- neutral
Adani Green Energy arm commissions India’s first wind-solar hybrid plant in Rajasthan ---- neutral
#######################################################
Buy Adani Ports and SEZ ; target of Rs 900: ICICI Direct ---- neutral
Adani Ports Q4 Review - Embarking On Another Round Of Capex: ICICI Direct ---- neutral


In [17]:
def get_sent(val):
    agg = 0
    for i in val:
        if i == 'positive':
            agg = agg + 1
        elif i == 'negative':
            agg = agg - 1

    if agg > 0:
        return('positive')
    elif agg < 0:
        return('negative')
    else:
        return('neutral')

In [18]:
sent = list()
for i in tot_val:
    sent.append(get_sent(i))

In [19]:
sent

['neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'negative',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'negative',
 'pos

In [46]:
tick_df['sentiment'] = sent 

tick_df.tail(50)

,stock,tick,sentiment
50,ITC Ltd,ITC,positive
51,Indian Oil Corporation Ltd,IOC,neutral
52,Indian Railway Catering and Tourism Corporatio...,INIR,positive
53,Indus Towers Ltd,BHRI,neutral
54,Indusind Bank Ltd,INBK,positive
55,Info Edge (India) Ltd,INED,negative
56,Infosys Ltd,INFY,positive
57,Interglobe Aviation Ltd,INGL,neutral
58,JSW Steel Ltd,JSTL,neutral
59,Kotak Mahindra Bank Ltd,KTKM,neutral


In [25]:
from google.colab import files
import io
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['nifty50_mcwb.csv']))
df.head()


Saving nifty50_mcwb.csv to nifty50_mcwb.csv


,Annexure II- Nifty 50 Index : Apr 2022,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sr. No,Security Symbol,Security Name,Industry,Equity Capital (In Rs.),Free Float Market Capitalisation (Rs. Crores),Weightage (%),Beta,R2,Volatility (%),Monthly Return,Avg. Impact Cost (%)
2,1,ADANIPORTS,Adani Ports and Special Economic Zone Ltd.,PORT,4224746460,61507.24,0.82,1.37,0.34,2.63,10.62,0.03
3,2,APOLLOHOSP,Apollo Hospitals Enterprise Ltd.,HOSPITAL,718923285,45473.17,0.61,0.83,0.11,2.15,-1.37,0.02
4,3,ASIANPAINT,Asian Paints Ltd.,PAINTS,959197790,145940.41,1.95,0.75,0.20,1.46,5.11,0.02


In [26]:
new_header = df.iloc[1] #grab the first row for the header
df = df[2:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df.head()


1,Sr. No,Security Symbol,Security Name,Industry,Equity Capital (In Rs.),Free Float Market Capitalisation (Rs. Crores),Weightage (%),Beta,R2,Volatility (%),Monthly Return,Avg. Impact Cost (%)
2,1,ADANIPORTS,Adani Ports and Special Economic Zone Ltd.,PORT,4224746460,61507.24,0.82,1.37,0.34,2.63,10.62,0.03
3,2,APOLLOHOSP,Apollo Hospitals Enterprise Ltd.,HOSPITAL,718923285,45473.17,0.61,0.83,0.11,2.15,-1.37,0.02
4,3,ASIANPAINT,Asian Paints Ltd.,PAINTS,959197790,145940.41,1.95,0.75,0.20,1.46,5.11,0.02
5,4,AXISBANK,Axis Bank Ltd.,PRIVATE SECTOR BANK,6139220726,192340.56,2.57,1.11,0.39,1.95,-4.28,0.02
6,5,BAJAJ-AUTO,Bajaj Auto Ltd.,MOTOR CYCLES/SCOOTERS,2893670200,48564.39,0.65,0.68,0.22,1.85,2.10,0.03


In [30]:
df2 = df.iloc[:50]
df2.head()

1,Sr. No,Security Symbol,Security Name,Industry,Equity Capital (In Rs.),Free Float Market Capitalisation (Rs. Crores),Weightage (%),Beta,R2,Volatility (%),Monthly Return,Avg. Impact Cost (%)
2,1,ADANIPORTS,Adani Ports and Special Economic Zone Ltd.,PORT,4224746460,61507.24,0.82,1.37,0.34,2.63,10.62,0.03
3,2,APOLLOHOSP,Apollo Hospitals Enterprise Ltd.,HOSPITAL,718923285,45473.17,0.61,0.83,0.11,2.15,-1.37,0.02
4,3,ASIANPAINT,Asian Paints Ltd.,PAINTS,959197790,145940.41,1.95,0.75,0.20,1.46,5.11,0.02
5,4,AXISBANK,Axis Bank Ltd.,PRIVATE SECTOR BANK,6139220726,192340.56,2.57,1.11,0.39,1.95,-4.28,0.02
6,5,BAJAJ-AUTO,Bajaj Auto Ltd.,MOTOR CYCLES/SCOOTERS,2893670200,48564.39,0.65,0.68,0.22,1.85,2.10,0.03


In [31]:
df3 = pd.DataFrame()
df3["tick"] = df2['Security Symbol']

df3['Weight']=df2['Weightage (%)']

In [44]:
df4 = df3.reset_index(drop=True)
df4.head(50)

,tick,Weight
0,ADANIPORTS,0.82
1,APOLLOHOSP,0.61
2,ASIANPAINT,1.95
3,AXISBANK,2.57
4,BAJAJ-AUTO,0.65
5,BAJFINANCE,2.37
6,BAJAJFINSV,1.20
7,BPCL,0.46
8,BHARTIARTL,2.33
9,BRITANNIA,0.52


In [ ]:
#Adani Ports and Special Economic Zone Ltd	APSE	= ADANIPORTS
#Apollo Hospitals Enterprise Ltd	APLH	= APOLLOHOSP
#



In [47]:
# Note tick labels are different for the same company
# Convert all tick labels indentical in both data frames

df4.loc[0,'tick']='APSE'
df4.loc[1,'tick']='APLH'
df4.loc[2,'tick']='ASPN'
df4.loc[3,'tick']='AXBK'
df4.loc[4,'tick']='BAJA'
df4.loc[5,'tick']='BJFN'
df4.loc[6,'tick']='BJFS'
df4.loc[7,'tick']='BPCL' # names match
df4.loc[8,'tick']='BRTI'
df4.loc[9,'tick']='BRIT'
df4.loc[10,'tick']='CIPL'
df4.loc[11,'tick']='COAL'
df4.loc[12,'tick']='DIVI'
df4.loc[13,'tick']='REDY'
df4.loc[14,'tick']='EICH'
df4.loc[15,'tick']='GRAS'
df4.loc[16,'tick']='HCLT'
df4.loc[17,'tick']='HDBK'
df4.loc[18,'tick']='HDFL'
df4.loc[19,'tick']='HROM'
df4.loc[20,'tick']='HALC'
df4.loc[21,'tick']='HLL'
df4.loc[22,'tick']='HDFC' # NAMES MATCH
df4.loc[23,'tick']='ICBK'
df4.loc[24,'tick']='ITC' #NAMES MATCH
df4.loc[25,'tick']='INBK'
df4.loc[26,'tick']='INFY' # NAMES MATCH
df4.loc[27,'tick']='JSTL'
df4.loc[28,'tick']='KTKM'
df4.loc[29,'tick']='LART'
df4.loc[30,'tick']='MAHM'
df4.loc[31,'tick']='MRTI'
df4.loc[32,'tick']='NTPC' # NAMES MATCH
df4.loc[33,'tick']='NEST'
df4.loc[34,'tick']='ONGC' # MATCH
df4.loc[35,'tick']='PGRD'
df4.loc[36,'tick']='RELI'
df4.loc[37,'tick']='SBIL'
df4.loc[38,'tick']='SHCM'
df4.loc[39,'tick']='SBI'
df4.loc[40,'tick']='SUN'
df4.loc[41,'tick']='TCS' # MATCH
df4.loc[42,'tick']='TACN'
df4.loc[43,'tick']='TAMdv'
df4.loc[44,'tick']='TISC'
df4.loc[45,'tick']='TEML'
df4.loc[46,'tick']='TITN'
df4.loc[47,'tick']='UPLL'
df4.loc[48,'tick']='ULTC'
df4.loc[49,'tick']='WIPR'

df4.head(50)

,tick,Weight
0,APSE,0.82
1,APLH,0.61
2,ASPN,1.95
3,AXBK,2.57
4,BAJA,0.65
5,BJFN,2.37
6,BJFS,1.20
7,BPCL,0.46
8,BRTI,2.33
9,BRIT,0.52


In [49]:
df5=tick_df.merge(df4, how='inner', on='tick')
df5.head(50)

,stock,tick,sentiment,Weight
0,Adani Ports and Special Economic Zone Ltd,APSE,positive,0.82
1,Apollo Hospitals Enterprise Ltd,APLH,neutral,0.61
2,Asian Paints Ltd,ASPN,positive,1.95
3,Axis Bank Ltd,AXBK,positive,2.57
4,Bajaj Auto Ltd,BAJA,neutral,0.65
5,Bajaj Finance Ltd,BJFN,positive,2.37
6,Bajaj Finserv Ltd,BJFS,positive,1.20
7,Bharat Petroleum Corporation Ltd,BPCL,positive,0.46
8,Bharti Airtel Ltd,BRTI,neutral,2.33
9,Britannia Industries Ltd,BRIT,neutral,0.52


In [50]:
df5['Weight'] = df5['Weight'].astype(float)
sum=0
for i in range(len(df5)):
  if(df5.loc[i,'sentiment']=='positive'):
    fac=1.0
  if(df5.loc[i,'sentiment']=='negative'):
    fac=-1.0
  if(df5.loc[i,'sentiment']=='neutral'):
    fac=0.0
  sum = sum+fac*df5.loc[i,'Weight']

sum

60.46